                                               Funciones para la API:

In [412]:
import pandas as pd
import scipy as sp
import numpy as np
import pyarrow.parquet as pq
import pyarrow as pa
from sklearn.metrics.pairwise import cosine_similarity

#Se cargan los datos
DF_GAMES = pd.read_parquet('../Parquets/ETL_Steam_Games.parquet')
DF_REVIEWS = pd.read_parquet('../Parquets/Review_Sentiment_Analysis.parquet')
DF_ITEMS = pd.read_parquet('../Parquets/ETL_user_items.parquet')
recomendaciones = pd.read_parquet('../Parquets/Recomendacion.parquet')


In [351]:
DF_GAMES.head()

,Publisher,Genres,App_Name,Price,Item_Id,Developer,Release_Year
0,Kotoshiro,"Action,Casual,Indie,Simulation,Strategy",Lost Summoner Kitty,4.99,761140,Kotoshiro,2018
1,"Making Fun, Inc.","Free to Play,Indie,RPG,Strategy",Ironbound,0.00,643980,Secret Level Srl,2018
2,彼岸领域,"Action,Adventure,Casual",弹炸人2222,0.99,767400,,2017
3,Trickjump Games Ltd,"Action,Adventure,Simulation",Battle Royale Trainer,3.99,772540,Trickjump Games Ltd,2018
4,Poppermost Productions,"Free to Play,Indie,Simulation,Sports",SNOW - All Access Basic Pass,9.99,774276,Poppermost Productions,2018


Ya realizamos el EDA,y observamos algunos datos sobre los publicadores y el precio de los juegos, sin embargo ahora no sean de utilidad para las funciones, por lo que se eliminaran, aumentando ademas la eficiencia.

In [352]:
DF_GAMES.head()

,Publisher,Genres,App_Name,Price,Item_Id,Developer,Release_Year
0,Kotoshiro,"Action,Casual,Indie,Simulation,Strategy",Lost Summoner Kitty,4.99,761140,Kotoshiro,2018
1,"Making Fun, Inc.","Free to Play,Indie,RPG,Strategy",Ironbound,0.00,643980,Secret Level Srl,2018
2,彼岸领域,"Action,Adventure,Casual",弹炸人2222,0.99,767400,,2017
3,Trickjump Games Ltd,"Action,Adventure,Simulation",Battle Royale Trainer,3.99,772540,Trickjump Games Ltd,2018
4,Poppermost Productions,"Free to Play,Indie,Simulation,Sports",SNOW - All Access Basic Pass,9.99,774276,Poppermost Productions,2018


In [353]:
DF_ITEMS.head()

,Item_Id,Item_Name,Playtime_Forever,User_Id
0,10,Counter-Strike,6,76561197970982479
1,20,Team Fortress Classic,0,76561197970982479
2,30,Day of Defeat,7,76561197970982479
3,40,Deathmatch Classic,0,76561197970982479
4,50,Half-Life: Opposing Force,0,76561197970982479


In [354]:
DF_REVIEWS.head()

,User_Id,Item_Id,Recommend,Year_Posted,Sentiment_Analysis
0,76561197970982479,1250,True,2011,2
1,76561197970982479,22200,True,2011,2
2,76561197970982479,43110,True,2011,2
3,js41637,251610,True,2014,2
4,js41637,227300,True,2013,2


Con los dataframes listos, se crea una tabla que será de utilidad para las funciones y que busca aumentar la eficiencia a la hora de deployar laAPI:

In [355]:
DF_MERGE = DF_ITEMS.merge(DF_REVIEWS, on=('User_Id', 'Item_Id'))
DF_MERGE.head()

,Item_Id,Item_Name,Playtime_Forever,User_Id,Recommend,Year_Posted,Sentiment_Analysis
0,22200,Zeno Clash,271,76561197970982479,True,2011,2
1,1250,Killing Floor,10006,76561197970982479,True,2011,2
2,43110,Metro 2033,834,76561197970982479,True,2011,2
3,227300,Euro Truck Simulator 2,551,js41637,True,2013,2
4,239030,"Papers, Please",349,js41637,True,2013,2


In [356]:
#Realizamos un merge entre el merge anterior y "user_games"
DF_MERGE = DF_MERGE.merge(DF_GAMES, on='Item_Id')
#Visualizamos los cambios
DF_MERGE.head()

,Item_Id,Item_Name,Playtime_Forever,User_Id,Recommend,Year_Posted,Sentiment_Analysis,Publisher,Genres,App_Name,Price,Developer,Release_Year
0,239030,"Papers, Please",349,js41637,True,2013,2,3909,"Adventure,Indie","Papers, Please",9.99,,2013
1,239030,"Papers, Please",250,SirParadiseLost,True,2015,2,3909,"Adventure,Indie","Papers, Please",9.99,,2013
2,239030,"Papers, Please",264,doctorpotatoes,True,2015,2,3909,"Adventure,Indie","Papers, Please",9.99,,2013
3,239030,"Papers, Please",813,commandochou,True,2014,1,3909,"Adventure,Indie","Papers, Please",9.99,,2013
4,239030,"Papers, Please",697,76561198063199558,True,2014,0,3909,"Adventure,Indie","Papers, Please",9.99,,2013


In [357]:
DF_MERGE.shape

(9199, 13)

Se chequea que todas las columnas tengan el tipo de dato correcto

In [358]:
DF_MERGE.dtypes

Item_Id                 int32
Item_Name              object
Playtime_Forever        int64
User_Id                object
Recommend                bool
Year_Posted             int64
Sentiment_Analysis      int64
Publisher              object
Genres                 object
App_Name               object
Price                 float64
Developer              object
Release_Year            int32
dtype: object

Observando las funciones solicitadas para la API, las columnas que se necesitarán son:

    Item_Id
    Playtime_Forever
    User_Id
    Genres
    Year_Release
    Recommend
    Developer
    Year_Posted
    Release_Year
    Item_Name



In [359]:
DF_MERGE = DF_MERGE.drop(['Publisher','Price','App_Name'], axis=1)
DF_MERGE.head()

,Item_Id,Item_Name,Playtime_Forever,User_Id,Recommend,Year_Posted,Sentiment_Analysis,Genres,Developer,Release_Year
0,239030,"Papers, Please",349,js41637,True,2013,2,"Adventure,Indie",,2013
1,239030,"Papers, Please",250,SirParadiseLost,True,2015,2,"Adventure,Indie",,2013
2,239030,"Papers, Please",264,doctorpotatoes,True,2015,2,"Adventure,Indie",,2013
3,239030,"Papers, Please",813,commandochou,True,2014,1,"Adventure,Indie",,2013
4,239030,"Papers, Please",697,76561198063199558,True,2014,0,"Adventure,Indie",,2013


In [360]:
DF_MERGE['Playtime_Forever'] = DF_MERGE['Playtime_Forever'] / 60

In [361]:
DF_MERGE.head()

,Item_Id,Item_Name,Playtime_Forever,User_Id,Recommend,Year_Posted,Sentiment_Analysis,Genres,Developer,Release_Year
0,239030,"Papers, Please",5.816667,js41637,True,2013,2,"Adventure,Indie",,2013
1,239030,"Papers, Please",4.166667,SirParadiseLost,True,2015,2,"Adventure,Indie",,2013
2,239030,"Papers, Please",4.400000,doctorpotatoes,True,2015,2,"Adventure,Indie",,2013
3,239030,"Papers, Please",13.550000,commandochou,True,2014,1,"Adventure,Indie",,2013
4,239030,"Papers, Please",11.616667,76561198063199558,True,2014,0,"Adventure,Indie",,2013


Guardamos este dataframe como parquet.

In [362]:
DF_MERGE.to_parquet('tabla_funciones.parquet')

In [363]:
tabla_funciones = pd.read_parquet('tabla_funciones.parquet')
tabla_funciones.head(2)

,Item_Id,Item_Name,Playtime_Forever,User_Id,Recommend,Year_Posted,Sentiment_Analysis,Genres,Developer,Release_Year
0,239030,"Papers, Please",5.816667,js41637,True,2013,2,"Adventure,Indie",,2013
1,239030,"Papers, Please",4.166667,SirParadiseLost,True,2015,2,"Adventure,Indie",,2013


Vamos a crear subtablas para cada funcion, con el fin, nuevamente, de optimizar el espacio usadoen render:

In [364]:
#Columnas a usar para la fucion 3:
F3= tabla_funciones[['Item_Name','Year_Posted', 'Recommend', 'Sentiment_Analysis']]

In [365]:
#Se buscan y eliminan posibles duplicados:
F3 = F3.drop_duplicates(subset=['Item_Name','Year_Posted', 'Recommend', 'Sentiment_Analysis'])
F3 = F3.reset_index(drop=True)
F3.to_parquet('F3.parquet')

In [366]:
#Columnas para la funcion 4:
F4= tabla_funciones[['Developer','Year_Posted', 'Recommend', 'Sentiment_Analysis']]

In [367]:
#Se buscan y eliminan posibles duplicados:
F4 = F4.drop_duplicates(subset=['Developer','Year_Posted', 'Recommend', 'Sentiment_Analysis'])
F4 = F4.reset_index(drop=True)
F4.to_parquet('F4.parquet')

In [368]:
#Columnas para la funcion 4:
F5= tabla_funciones[['Developer', 'Sentiment_Analysis']]

In [369]:
#Se buscan y eliminan posibles duplicados:
F5 = F5.drop_duplicates(subset=['Developer', 'Sentiment_Analysis'])
F5 = F5.reset_index(drop=True)
F5.to_parquet('F5.parquet')

                                            FUNCION 1

In [370]:
def PlayTimeGenre(genero: str):
    #Se filtra por genero:
    dfgenero = tabla_funciones[tabla_funciones['Genres'].str.contains(genero, case=False, na=False)]

   #Se agrupa y sumar horas jugadas
    max_year = dfgenero.groupby('Release_Year')['Playtime_Forever'].sum().idxmax()


    result = {"Año de lanzamiento con más horas jugadas para Género " + genero: int(max_year)}
    return result

In [371]:
PLAYTIME = PlayTimeGenre('Action')
PLAYTIME

{'Año de lanzamiento con más horas jugadas para Género Action': 2013}

                                            FUNCION 2

In [372]:
def UserForGenre(genero: str):
    dfgenero2 = tabla_funciones[tabla_funciones['Genres'].str.contains(genero, case=False, na=False)]
#Se agrupa por usuario y se suma las horas jugadas
    user_time = dfgenero2.groupby('User_Id')['Playtime_Forever'].sum()
#Se busca el usuario con más horas jugadas
    maxtime = user_time.idxmax()
#Se obtienr la acumulación de horas jugadas por año
    playtime_year = dfgenero2.groupby('Release_Year')['Playtime_Forever'].sum().reset_index() 
    result = {"Usuario con más horas jugadas para Género " + genero: maxtime,
              "Horas jugadas": playtime_year}
    return result


In [373]:
GENRE_HOURS =UserForGenre('Casual')
GENRE_HOURS

{'Usuario con más horas jugadas para Género Casual': '76561198075267655',
 'Horas jugadas':     Release_Year  Playtime_Forever
 0           1992          0.833333
 1           1993          0.850000
 2           1996         16.133333
 3           1998          2.916667
 4           1999          1.000000
 5           2006          2.883333
 6           2007        369.783333
 7           2009        110.200000
 8           2010        275.883333
 9           2011         56.633333
 10          2012       5745.283333
 11          2013       2418.400000
 12          2014      10066.416667
 13          2015      10498.216667
 14          2016       2241.466667
 15          2017      58197.566667}

                                                FUNCION 3

In [374]:
def UsersRecommend(año: int):
    '''Recordemos, que esta funcion solo retorna la lista para los años 2010 a 2015'''
    #Se filtra el DataFrame para solo incluir las filas del año dado,
    # donde el juego fue recomendado y el análisis de sentimiento es neutral o positivo.
    df_filtered = F3[(F3['Year_Posted'] == año) & (F3['Recommend'] == True) & (F3['Sentiment_Analysis'] >= 1)]
    
    #Se agrupa por nombre de juego y cuenta las recomendaciones para cada juego.
    game_counts = df_filtered['Item_Name'].value_counts()
    
    #Top tres juegos más recomendados.
    top_3 = game_counts[:3].index.tolist()
    result = [{"Puesto 1": top_3[0]}, {"Puesto 2": top_3[1]}, {"Puesto 3": top_3[2]}]
    
    return result

In [375]:
MOST_RECO = UsersRecommend(2012)
MOST_RECO

[{'Puesto 1': 'Day of Defeat: Source'},
 {'Puesto 2': 'Counter-Strike: Source'},
 {'Puesto 3': 'Call of Duty: Black Ops'}]

                                                FUNCION 4

In [376]:

def UsersWorstDeveloper(año: int):
    # Filtrar el DataFrame para solo incluir las filas del año dado,
    # donde el juego NO fue recomendado y el análisis de sentimiento es negativo.
    df_filtered = F4[(F4['Year_Posted'] == año) & (F4['Recommend'] == False) & (F4['Sentiment_Analysis'] == 0)]
    
    # Agrupar por desarrolladora y contar las no recomendaciones para cada una.
    developer_counts = df_filtered['Developer'].value_counts()
    
    # Obtener las tres desarrolladoras con más juegos no recomendados.
    worst_dev = developer_counts[:3].index.tolist()
    
    # Crear la lista de diccionarios para devolver.
    result = []
    if len(worst_dev) > 0: result.append({"Puesto 1": worst_dev[0]})
    if len(worst_dev) > 1: result.append({"Puesto 2": worst_dev[1]})
    if len(worst_dev) > 2: result.append({"Puesto 3": worst_dev[2]})
    
    return result

In [377]:
LEAST_RECO = UsersWorstDeveloper(2015)
LEAST_RECO

[{'Puesto 1': 'Ronimo Games'},
 {'Puesto 2': 'Dark Day Interactive'},
 {'Puesto 3': 'Bitmonster Inc'}]

                                              FUNCION 5

In [378]:
def sentiment_analysis(desarrolladora: str):   
    #Se cuenta la cantidad de registros de reseñas por categoría de sentimiento
    sentiment_counts = F5['Sentiment_Analysis'].value_counts().to_dict()
    
    #Se crea el diccionario de salida
    output_dict = {desarrolladora: {'Negative': sentiment_counts.get(0, 0),
                                    'Neutral': sentiment_counts.get(1, 0),
                                    'Positive': sentiment_counts.get(2, 0)}}
    return output_dict

In [379]:
SENTIMENT_DEVS = sentiment_analysis('SEGA')
SENTIMENT_DEVS

{'SEGA': {'Negative': 320, 'Neutral': 309, 'Positive': 581}}

                                    SISTEMA RECOMENDACION  ITEM-ITEM

In [380]:
recomendaciones.head()

,User_Id,Item_Name,Rating
0,76561197970982479,Killing Floor,5
1,76561197970982479,Zeno Clash,5
2,76561197970982479,Metro 2033,5
3,js41637,Barbie™ Dreamhouse Party™,5
4,js41637,Euro Truck Simulator 2,5


Se inicia creando una matriz de recomendaciones a partir del archivo recomendacion.parquet, dónde sus filas son los ususarios, las columnas los juegos, y los datos las calificaciones respectivas;a continuación, se realiza su normalización y preprocesamiento para eliminar columnas sin calificaciones.

In [392]:
#matriz user item calif
matrix = recomendaciones.pivot(index='User_Id', columns='Item_Name', values='Rating')
# Normalización del dataframe 'matrix'
matrix_norm = matrix.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)
# Se borran las columnas que contienen solo cero o no tienen rating, se rellenan los vacios con 0 y se hace la transpuesta
matrix_norm.fillna(0, inplace=True)
matrix_norm = matrix_norm.T
matrix_norm = matrix_norm.loc[:, (matrix_norm != 0).any(axis=0)]
matrix_norm

User_Id,-2SV-vuLB-Kg,-GM-Dragon,-PRoSlayeR-,-SEVEN-,-_PussyDestroyer_-,0-3-0,00000000000000000001227,00True,01189958889189157253,01221733,...,zixwot,zombi_anon,zomgieee,zoozles,zrustz16,zsharoarkbr,zuilde,zuzuga2003,zv_odd,zyr0n1c
Item_Name,,,,,,,,,,,,,,,,,,,,,
0RBITALIS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
"10,000,000",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
100% Orange Juice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
1001 Spikes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
12 Labours of Hercules,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
resident evil 4 / biohazard 4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
sZone-Online,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
the static speaks my name,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0


Posteriormente se construye la matriz dispersa (sparse matrix) buscando disminuir el uso de memoria y mejorar el manejo de conjuntos de datos extensos, especialmente cuando la mayoría de los valores en la matriz son cero.Esta matriz es una estructura de datos que almacena únicamente los valores diferentes de cero junto con sus posiciones en la matriz, en lugar de almacenar todos los valores.

In [393]:
#matriz dispersa
matrix_sparse = sp.sparse.csr_matrix(matrix_norm)
Item_sim = cosine_similarity(matrix_sparse)

Se calculan las similitudes entre elementos utilizando la similitud del coseno. Se crea el DataFrame de similitud, Item_sim_df, que será usado en la funcion de recomendacion.

In [396]:
#item similarity dataframe
Item_sim_df = pd.DataFrame(Item_sim, index = matrix_norm.index, columns = matrix_norm.index)s

In [409]:
def recomendacion_juego(id_producto):
    count = 1
    print('Similar games to {} include:\n'.format(id_producto))
    for item in Item_sim_df.sort_values(by = id_producto, ascending = False).index[1:6]:
        print(f"{''.join(item)}")
        count +=1  

In [410]:
# ejemplo
recomendacion_juego('Killing Floor')

Similar games to Killing Floor include:

Game of Thrones 
Peggle Deluxe
Dev Guy
Deus Ex: Revision
Shank


Una vez verificado el funcionamiento de la funcion de recomendacion, se guardan la matriz normalizada y nuestra matriz de similitud:


In [418]:
pq.write_table(pa.Table.from_pandas(matrix_norm), 'matrix_norm.parquet')
pq.write_table(pa.Table.from_pandas(Item_sim_df), 'item_sim_df.parquet')
